In [1]:
! pip install hazm

In [88]:
from __future__ import unicode_literals
from hazm import *


text = 'یک خودرو با طول دراز و عرض کوتاه از ما سبقت گرفت.'

tagger = POSTagger(model='resources/hazm/postagger.model')
tagger.tag(word_tokenize(text))

[('یک', 'NUM'),
 ('خودرو', 'N'),
 ('با', 'P'),
 ('طول', 'Ne'),
 ('دراز', 'AJ'),
 ('و', 'CONJ'),
 ('عرض', 'Ne'),
 ('کوتاه', 'AJ'),
 ('از', 'P'),
 ('ما', 'PRO'),
 ('سبقت', 'N'),
 ('گرفت', 'V'),
 ('.', 'PUNC')]

In [23]:
p = Normalizer()
p.normalize('جابهجایی')

'جابهجایی'

In [65]:
chunker = Chunker(model='resources/hazm/chunker.model')
tagged = tagger.tag(word_tokenize(text))
a = chunker.parse(tagged)
b = tree2brackets(a)
b

'[یک خودرو NP] [با PP] [سرعت NP] [کند VP] [از PP] [ما NP] [سبقت گرفت VP] .'

In [25]:
c = a[0]
print(1)

1


In [26]:
d = ' '.join([s[0] for s in list(a[0])])

In [27]:
d

'سرعت بالای خودروی عمه علی'

In [29]:
list(a)

[Tree('NP', [('سرعت', 'Ne'), ('بالای', 'AJe'), ('خودروی', 'Ne'), ('عمه', 'N'), ('علی', 'N')]),
 Tree('ADJP', [('عجیب', 'AJ')]),
 Tree('VP', [('است', 'V')]),
 ('.', 'PUNC')]

In [92]:
from __future__ import unicode_literals
from hazm import *
import re


tagger = POSTagger(model='resources/hazm/postagger.model')
chunker = Chunker(model='resources/hazm/chunker.model')


def join_patterns(patterns, grouping=False):
    return '(' + ('?:' if not grouping else '') + '|'.join(patterns) + ')'


def extract_qualitative(text, quantity, last_span):
    tagged = tagger.tag(word_tokenize(text))
    res = list(chunker.parse(tagged))
    for tree in res:
        tokens = list(tree)
        for idx in range(len(tokens)):
            if tokens[idx][0] == quantity and idx != len(tokens) - 1:
                chunk_str = ' '.join([s[0] for s in tokens])
                match_span = re.search(chunk_str, text).span()
                if tokens[idx+1][1].startswith('AJ') and match_span[0] > last_span:
                    return chunk_str



inverse_alternative = {}
with open('resources/dataset/inverse_alternative.json', 'r', encoding='utf-8') as fp:
    inverse_alternative = json.load(fp)

def qualitive_descriptions(text):
    sents = sent_tokenize(text)
    PATTERN = join_patterns(list(inverse_alternative.keys()))
    res = []
    read_chars = 0
    for sent in sents:
        last_span = 0
        for match in re.finditer(PATTERN, sent):
            marker = extract_qualitative(sent, match.group(), last_span)
            if marker is None:
                continue
            temp_dict = {
                'type': inverse_alternative[match.group()],
                'amount': '',
                'unit': '',
                'item': '',
                'marker': marker,
                'span': [match.span()[0] + read_chars, match.span()[0] + len(marker) + read_chars]
            }
            last_span = match.span()[0] + len(marker)
            res.append(temp_dict)
        read_chars += len(sent)
        
    return res
        

In [95]:
text = 'یک خودرو با طول دراز و عرض کوتاه از ما سبقت گرفت.'

b = 5 * text

b = p.normalize(b)

qualitive_descriptions(b)

[{'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول دراز و عرض کوتاه',
  'span': [12, 32]},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول دراز و عرض کوتاه',
  'span': [61, 81]},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول دراز و عرض کوتاه',
  'span': [110, 130]},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول دراز و عرض کوتاه',
  'span': [159, 179]},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول دراز و عرض کوتاه',
  'span': [208, 228]}]

In [31]:
import json
quantities = {}
with open('resources/dataset/quantities_related_names.json', 'r', encoding='utf-8') as fp:
    quantities = json.load(fp)

temp = {}

for k, v in quantities.items():
    for alt in v:
        temp[alt] = k

with open('resources/dataset/inverse_alternative.json', 'w', encoding='utf-8') as fp:
    json.dump(temp, fp, indent=4)

In [22]:
a[0][0]

('سرعت', 'Ne')

In [26]:
import json
quantities = {}
with open('resources/dataset/quantities_related_names.json', 'r', encoding='utf-8') as fp:
    quantities = json.load(fp)

quantities_names = []
for k, v in quantities.items():
    quantities_names.extend(v)

quantities_names

with open('resources/dataset/alternatives_for_quantities.json', 'w', encoding='utf-8') as fp:
    json.dump(quantities_names, fp, indent=4)

In [27]:
alternatives = {}
with open('resources/dataset/alternatives_for_quantities.json', 'r', encoding='utf-8') as fp:
    alternatives = json.load(fp)

alternatives

['تغییرات سرعت به تغییرات زمان',
 'شتاب',
 'تغییرات سرعت نسبت به تغییرات زمان',
 'تغییرات سرعت نسبت به زمان',
 'گشتاور دورانی',
 'گشتاور زاویه\u200cای',
 'تکانه زاویه\u200cای',
 'سطح',
 'زمین',
 'مساحت',
 'چگالی سطح',
 'چگالی سطحی',
 'جرم سطحی',
 'ظرفیت',
 'خازن',
 'کاپاسیتانس',
 'پتانسیل شیمیایی',
 'آنتروپی',
 'آنتالپی',
 'چگالی جریان',
 'جریان الکتریکی بر واحد سطح',
 'بار الکتریکی',
 'بار',
 'الکتریسیته',
 'پتانسیل الکتریکی',
 'ولتاژ',
 'پتانسل',
 'اختلاف پتانسیل',
 'مقاومت الکتریکی',
 'مقاومت',
 'رزیستانس',
 'انرژی',
 'گرما',
 'ارزش غذایی',
 'کارمایه',
 'پتانسیل',
 'جنبشی',
 'آنتروپی',
 'آنتالپی',
 'توان',
 'نرخ',
 'فشار',
 'گشتاور',
 'نیروی چرخشی',
 'نیروی دورانی',
 'ممان نیرو',
 'سرعت',
 'تندی',
 'جابجایی نسبت به زمان',
 'مسافت نسبت به زمان',
 'جابجایی نسبت به تغییرات زمان',
 'مسافت نسبت به تغییرات زمان',
 'مسافت طی شده نسبت به تغییرات زمان',
 'حجم',
 'فضا',
 'طول',
 'فاصله',
 'عرض',
 'ارتفاع',
 'جابه\u200cجایی',
 'جا به جایی',
 'جابجایی',
 'مسافت',
 'بازه',
 'شعاع',
 'قطر',
 'ضلع

In [ ]:
class Extracted_quantity:

    def __init(self, type, amount, unit, item, marker, span, SI_equivalent):
        self.type = type
        self.amount = amount
        self.unit = unit
        self.item = item
        self.marker = marker
        self.span = span
        self.SI_equivalent = SI_equivalent

    def get_dict(self):
        return {
            'type': self.type,
            'amount': self.amount,
            'unit': self.unit,
            'item': self.item,
            'marker': self.marker,
            'span': self.span,
            'SI_equivalent': self.SI_equivalent,
        }

